# Dynamic Plotting Libraries for Python

In [ ]:
# imports
import numpy as np
import pandas as pd

# plotly
import plotly.express as px   # conda install plotly
import plotly.io as pio

# cufflinks
import cufflinks as cf   # pip install cufflinks
cf.go_offline()

# Animation demo
import time
from skimage import io

In [ ]:
# plotly template options

pio.templates

### plotly (+cufflinks)

Interactive charts and maps for Python, R, Julia, Javascript, ggplot2, F#, MATLAB®, and Dash.

https://plotly.com/graphing-libraries/

In [ ]:
# Using Bitcoin data

bitcoin = pd.read_csv('./datasets/bitcoin.csv')
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'], dayfirst=False)
bitcoin.info()

In [ ]:
bitcoin

In [ ]:
# Using plotly express
px.scatter(data_frame=bitcoin,
           x='Date',
           y='Close',
           labels={'Date':'Operation Date','Close':'Close Price (USD)'},
           title='Bitcoin price evolution')

#### [Cufflinks](https://github.com/santosjorge/cufflinks)

Cufflinks lets us generate interactive plots based on plotly directly from pandas dataframe by calling the `iplot()` method on it passing various parameters. 

However, documentation is not too good, therefore, you may check this [link](https://coderzcolumn.com/tutorials/data-science/cufflinks-how-to-create-plotly-charts-from-pandas-dataframe-with-one-line-of-code) out to watch some examples.

In [ ]:
# Comprehensive list of parameters

cf.help()

In [ ]:
# One data serie example

bitcoin.iplot(x='Date',
              y='Close',
              xTitle='Operation Date',
              yTitle='Close Price (USD)',
              title='Bitcoin price evolution');

In [ ]:
# Little problem with cufflinks

bitcoin = bitcoin.sort_values(by='Date')

In [ ]:
# Two data series example (same plot)

bitcoin.iplot(x='Date',
              y=['Open', 'Close'],
              xTitle='Operation Date',
              yTitle='Close Price (USD)',
              title='Bitcoin price evolution');

In [ ]:
# Two data series example (two plots)

bitcoin.iplot(x='Date',
              y=['Open', 'Close'],
              subplots=True,
              xTitle='Operation Date',
              yTitle='Close Price (USD)',
              title='Bitcoin price evolution');

In [ ]:
# More parameters...

bitcoin.iplot(x='Date',
              y=['Open', 'Close'],
              subplots=True,
              fill=True,
              colors=['green','red'],
              xTitle='Operation Date',
              yTitle='Close Price (USD)',
              title='Bitcoin price evolution');

---

In [ ]:
# Using Stroke data to create histograms

stroke_risk = pd.read_csv('./datasets/healthcare-dataset-stroke-data.csv')
print(stroke_risk.info())
stroke_risk

In [ ]:
# Stroke vs. No stroke cases dataframes

stroke_risk_true = stroke_risk[stroke_risk['stroke'] == 1]
stroke_risk_false = stroke_risk[stroke_risk['stroke'] == 0]
print('Stroke: ', stroke_risk_true.shape)
print('No stroke: ', stroke_risk_false.shape)

In [ ]:
# Histogram using cufflinks

feature = 'bmi'

stroke_risk_true[feature].iplot(kind='hist',
                                xTitle=f'Analysed feature: {feature}',
                                yTitle=f'No. of Cases',
                                title='Stroke Risk Analysis',
                                colors=['blue']);

In [ ]:
# Histogram using plotly express

px.histogram(data_frame=stroke_risk_true[feature],
             labels={'value':f'Analysed feature: {feature}'},
             title='Stroke Risk Analysis',
             template='plotly_white')

---

### Animation demo: [_Visualization of MRI volume slices_](https://plotly.com/python/visualizing-mri-volume-slices/)

In [ ]:
# Import data
vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(z=(6.7 - k * 0.1) * np.ones((r, c)),
                                                 surfacecolor=np.flipud(volume[67 - k]),
                                                 cmin=0, cmax=200),
                                 name=str(k)) for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(z=6.7 * np.ones((r, c)),
                         surfacecolor=np.flipud(volume[67]),
                         colorscale='Gray',
                         cmin=0, cmax=200,
                         colorbar=dict(thickness=20, ticklen=4)))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
    }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

---